# Get Planck MCMC parameter Covariance Matrices and Process into Fisher matrices

In [1]:
# load the data
planckDir = 'Planck_data/'

covmatFile1 = 'base_plikHM_EE_lowTEB.covmat'
paramNamesFile1 = 'base_plikHM_EE_lowTEB.paramnames'
covmatFile2 = 'base_plikHM_EE.covmat'
paramNamesFile2 = 'base_plikHM_EE.paramnames'
covmatFile3 = 'base_plikHM_TTTEEE_lowTEB.covmat'
paramNamesFile3 = 'base_plikHM_TTTEEE_lowTEB.paramnames'

# data from pages:
# https://wiki.cosmos.esa.int/planckpla2015/index.php/Cosmological_Parameters
# http://pla.esac.esa.int/pla/#cosmology

covmat1 = np.loadtxt(planckDir+covmatFile1)
text_file = open(planckDir+paramNamesFile1, "r")
paramNames1 = text_file.readlines()
#print paramNames1
print len(paramNames1)
text_file.close()

covmat2 = np.loadtxt(planckDir+covmatFile2)
text_file = open(planckDir+paramNamesFile2, "r")
paramNames2 = text_file.readlines()
#print paramNames2
print len(paramNames2)
text_file.close()

covmat3 = np.loadtxt(planckDir+covmatFile3)
text_file = open(planckDir+paramNamesFile3, "r")
paramNames3 = text_file.readlines()
#print paramNames3
print len(paramNames3)
text_file.close()

for lineNum in range(13):
    print 'line ',lineNum,': ',paramNames1[lineNum], \
                  '        : ',paramNames2[lineNum], \
                  '        : ',paramNames2[lineNum]

59
57
82
line  0 :  omegabh2	\Omega_b h^2
        :  omegabh2	\Omega_b h^2
        :  omegabh2	\Omega_b h^2

line  1 :  omegach2	\Omega_c h^2
        :  omegach2	\Omega_c h^2
        :  omegach2	\Omega_c h^2

line  2 :  theta	100\theta_{MC}
        :  theta	100\theta_{MC}
        :  theta	100\theta_{MC}

line  3 :  tau	\tau
        :  tau	\tau
        :  tau	\tau

line  4 :  logA	{\rm{ln}}(10^{10} A_s)
        :  logA	{\rm{ln}}(10^{10} A_s)
        :  logA	{\rm{ln}}(10^{10} A_s)

line  5 :  ns	n_s
        :  ns	n_s
        :  ns	n_s

line  6 :  calPlanck	y_{\rm cal}
        :  galfEE100	A^{{\rm dust}EE}_{100}
        :  galfEE100	A^{{\rm dust}EE}_{100}

line  7 :  galfEE100	A^{{\rm dust}EE}_{100}
        :  galfEE100143	A^{{\rm dust}EE}_{100\times 143}
        :  galfEE100143	A^{{\rm dust}EE}_{100\times 143}

line  8 :  galfEE100143	A^{{\rm dust}EE}_{100\times 143}
        :  galfEE100217	A^{{\rm dust}EE}_{100\times 217}
        :  galfEE100217	A^{{\rm dust}EE}_{100\times 217}

line  9

In [2]:
# get indices of interest from the above output
print 'covmat1 shape: ', covmat1.shape
print 'covmat2 shape: ', covmat2.shape
print 'covmat3 shape: ', covmat3.shape

# covariance matrix is only 20x20.  If I use this then I'll have to use theta_mc instead of H0.
selectIndices = [0,1,2,4,5,3] # used later to put them in order compatible with my Fij
print 'covmat1: ',[paramNames1[si] for si in selectIndices]
print 'covmat2: ',[paramNames2[si] for si in selectIndices]
print 'covmat3: ',[paramNames3[si] for si in selectIndices]



covmat1 shape:  (13, 13)
covmat2 shape:  (13, 13)
covmat3 shape:  (33, 33)
covmat1:  ['omegabh2\t\\Omega_b h^2\n', 'omegach2\t\\Omega_c h^2\n', 'theta\t100\\theta_{MC}\n', 'logA\t{\\rm{ln}}(10^{10} A_s)\n', 'ns\tn_s\n', 'tau\t\\tau\n']
covmat2:  ['omegabh2\t\\Omega_b h^2\n', 'omegach2\t\\Omega_c h^2\n', 'theta\t100\\theta_{MC}\n', 'logA\t{\\rm{ln}}(10^{10} A_s)\n', 'ns\tn_s\n', 'tau\t\\tau\n']
covmat3:  ['omegabh2\t\\Omega_b h^2\n', 'omegach2\t\\Omega_c h^2\n', 'theta\t100\\theta_{MC}\n', 'logA\t{\\rm{ln}}(10^{10} A_s)\n', 'ns\tn_s\n', 'tau\t\\tau\n']


In [3]:
# fix the nuissance parameters

# invert covariance matrices
invCov1 = np.linalg.inv(covmat1)
invCov2 = np.linalg.inv(covmat2)
invCov3 = np.linalg.inv(covmat3)

# print shapes and diagonals
print invCov1.shape
print invCov2.shape
print invCov3.shape
print np.array([invCov1[si,si] for si in selectIndices])
print np.array([invCov2[si,si] for si in selectIndices])
print np.array([invCov3[si,si] for si in selectIndices])

#subInvCov = [invCov[si1][si2] for si1 in selectIndices for si2 in selectIndices]
#subInvCov = [[invCov[:,si1] for si1 in selectIndices[si2]] for si2 in selectIndices]
#subInvCov1 = [invCov[si1] for si1 in selectIndices]
#print subInvCov1
#subInvCov = [subInvCov1[:,si2] for si2 in selectIndices]

nParams = selectIndices.__len__()
subInvCov1 = np.empty((nParams,nParams))
subInvCov2 = np.empty((nParams,nParams))
subInvCov3 = np.empty((nParams,nParams))
# pull out the parameters indicated in selectIndices in order indicated there
for si1 in range(6):
    for si2 in range(6):
        subInvCov1[si1,si2] = invCov1[selectIndices[si1],selectIndices[si2]]
        subInvCov2[si1,si2] = invCov2[selectIndices[si1],selectIndices[si2]]
        subInvCov3[si1,si2] = invCov3[selectIndices[si1],selectIndices[si2]]
#print subInvCov
print np.diag(subInvCov1)
print np.diag(subInvCov2)
print np.diag(subInvCov3)

(13, 13)
(13, 13)
(33, 33)
[14392681.1929545   1049528.32399962  1437035.41942155    46769.37259216
    15165.26890736   189447.056113  ]
[1.43899404e+07 1.08746731e+06 1.38833734e+06 4.72009326e+04
 1.22552384e+04 1.90637457e+05]
[93644638.67707175  4268555.25616327 14958494.88509028   804119.32282487
   369396.33712423  3204935.51385855]
[14392681.1929545   1049528.32399962  1437035.41942155    46769.37259216
    15165.26890736   189447.056113  ]
[1.43899404e+07 1.08746731e+06 1.38833734e+06 4.72009326e+04
 1.22552384e+04 1.90637457e+05]
[93644638.67707175  4268555.25616327 14958494.88509028   804119.32282487
   369396.33712423  3204935.51385855]


In [4]:
# get desired combinations before finding sigmas
"""
covmatFile1 = 'base_plikHM_EE_lowTEB.covmat'
paramNamesFile1 = 'base_plikHM_EE_lowTEB.paramnames'
covmatFile2 = 'base_plikHM_EE.covmat'
paramNamesFile2 = 'base_plikHM_EE.paramnames'
covmatFile3 = 'base_plikHM_TTTEEE_lowTEB.covmat'
paramNamesFile3 = 'base_plikHM_TTTEEE_lowTEB.paramnames'
"""
Fij_plikHM_EE_lowTEB = subInvCov1
Fij_plikHM_EE = subInvCov2
Fij_plikHM_TTTEEE_lowTEB = subInvCov3

Fij_plikHM_lowTEB = Fij_plikHM_EE_lowTEB - Fij_plikHM_EE

In [5]:
# re-invert and get sigmas
getSigmas = lambda Fij: np.sqrt(np.diag(np.linalg.inv(Fij)))
sigmas_TTTEEE_lowTEB = getSigmas(Fij_plikHM_TTTEEE_lowTEB)
sigmas_lowTEB = getSigmas(Fij_plikHM_lowTEB)
print sigmas_TTTEEE_lowTEB
print sigmas_lowTEB

[0.00013805 0.00139624 0.00032041 0.03085309 0.00389213 0.01601033]
[0.00315051 0.00444641 0.00440098 0.01898294 0.01669401 0.02325072]


In [6]:
# save that sub-Fisher matrix to use in another notebook
# I will want it in "Rebuild covar with noise"
#np.savetxt('PlanckInvCovFromMCMC.txt',subInvCov)
np.savetxt('Planck_TTTEEE_lowTEB_FromMCMC.txt',Fij_plikHM_TTTEEE_lowTEB)
np.savetxt('Planck_lowTEB_FromMCMC.txt',Fij_plikHM_lowTEB)

# Transform to H0 parameter via Jacobian

In [7]:
# can I get a jacobian to translate theta_mc to H0?
# yes.  Use discrete derivative with CAMB.  
# dtheta/dparams vector below from notebook "Rebuild covar with noise"

In [8]:
# first need to convert 100*theta_MC to theta_MC
mcIndex = 2
mcFac = 100
Fij_plikHM_TTTEEE_lowTEB[:,mcIndex] *= mcFac
Fij_plikHM_TTTEEE_lowTEB[mcIndex,:] *= mcFac
Fij_plikHM_lowTEB[:,mcIndex] *= mcFac
Fij_plikHM_lowTEB[mcIndex,:] *= mcFac

In [9]:
#dtheta/dparams
nCosParams = 6
# paramList after transforming to (H0, 10^9 As) from (cosmomc_theta, ln(10^10*As) )
paramList = ['ombh2','omch2','H0','10^9 As','ns','tau'] 
dthetas = [-0.024833399132992862, 0.010457065703734872, 2.9621955038288544e-05, 0.0, 0.0, 0.0]

# The only derivatives in the Jacobian are on the dtheta/dparam row.
mcIndex = 2
jacobian = np.diag(np.ones(nCosParams))
jacobian[mcIndex,:] = dthetas#*100
#jacobian[:,mcIndex] = dthetas#*100

# multiply these all by d{100thetamc}/d{thetamc} = 100
#jacobian[mcIndex,:] *= 100
#jacobian[:,mcIndex] *= 100

print jacobian

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-2.48333991e-02  1.04570657e-02  2.96219550e-05  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00]]


In [10]:
# jacobian-it-up
Fij_plikHM_TTTEEE_lowTEB = np.dot(jacobian.T,np.dot(Fij_plikHM_TTTEEE_lowTEB,jacobian))
Fij_plikHM_lowTEB = np.dot(jacobian.T,np.dot(Fij_plikHM_lowTEB,jacobian))

#print Fij_plikHM_TTTEEE_lowTEB
#print Fij_plikHM_lowTEB

# Transform to $10^9 As$ via Jacobian

In [11]:
# only one partial derivative needed
# d( ln(10^10*As)) / d(10^9*As) = 1/(10^9*As)*d(10*10^9*As)/d(10^9*As) = 1/(10^9*As)
TenToTheNineAs_fid = 2.2
AsIndex = 3
Fij_plikHM_TTTEEE_lowTEB[:,AsIndex] *= 1/TenToTheNineAs_fid
Fij_plikHM_TTTEEE_lowTEB[AsIndex,:] *= 1/TenToTheNineAs_fid
Fij_plikHM_lowTEB[:,AsIndex] *= 1/TenToTheNineAs_fid
Fij_plikHM_lowTEB[AsIndex,:] *= 1/TenToTheNineAs_fid

# Load  FisherMatrix objects that have the CMB-S4 fiducial parameters for combination with this result

In [12]:
# load saved Fobj
import FisherCl as fcl
import noiseCl as ncl
import crosspower as cp
import pickle

#filehandler = open('Fisher_7_6X4_w2_wan.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=False,
#filehandler = open('Fisher_7_6X4_w2n.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=False,
#filehandler = open('Fisher_7_6X4n.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=False,
#neutrino_hierarchy = 'normal', usePrimaryCMB = True; from FisherCl branch quickCl
# kludge for this one: told cp.getDNDZinterp to use modelDNDZ3; set cp.tophat beesBins=True
# biasByBin = False (uses sigma(b)/b), dw = 0.05; X2n version has noise included

#filehandler = open('Fisher_7_6X2_w2_wan.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=True,
#filehandler = open('Fisher_7_6X2_w2n.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=True,
#filehandler = open('Fisher_7_6X2n.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000, nz=3e5; nonlinear=True,
#neutrino_hierarchy = 'normal', usePrimaryCMB = True; from FisherCl branch quickCl
# kludge for this one: told cp.getDNDZinterp to use modelDNDZ3; set cp.tophat beesBins=True
# biasByBin = False (uses sigma(b)/b), dw = 0.05; X2n version has noise included

# this one has TE polarazation to l=5000, but only one galaxy bin
#filehandler = open('Fisher_4_1Yn.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000; nz=3e5; nonlinear=True,
#neutrino_hierarchy = 'normal', usePrimaryCMB = True; from FisherCl branch quickCl
# this one used fiducial params and values to match CMB-S4 Science book verification table 8.1

# this on like the Yn version but uses 'total' rather than 'unlensed' power spectra
filehandler = open('Fisher_4_1Zn.obj','r') # AccuracyBoost=3, lmax=3000, lmaxP=5000; nz=3e5; nonlinear=True,
#neutrino_hierarchy='normal', usePrimaryCMB=True, nMapsP=3, useNoise=True; myClName='total'; 
#from FisherCl branch quickCl
# this one used fiducial params and values to match CMB-S4 Science book verification table 8.1

Fobj = pickle.load(filehandler) 

print Fobj.paramList
print Fobj.cosParams

IMPORT ERROR: /opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/quicklens/mpi.pyc (No module named pypar). Could not load pbs or pypar. MPI will not be used.


['ombh2', 'omch2', 'H0', 'As', 'ns', 'tau', 'mnu', 'w', 'wa', 'bin1']
{'tau': 0.06, 'r': 0, 'num_massive_neutrinos': 1, 'wa': 0.0, 'kPivot': 0.05, 'neutrino_hierarchy': 'normal', 'H0': 69, 'mnu': 0.06, 'omk': 0, 'standard_neutrino_neff': 3.046, 'As': 2.2e-09, 'omch2': 0.12, 'w': -1.0, 'cosmomc_theta': None, 'nnu': 3.046, 'ns': 0.966, 'ombh2': 0.0222}


In [13]:
# get TE fisher
myLmin = 30
myFij = Fobj.makeFisher(myLmin,TE=True)

# fix w, mnu 
# nCosParams should be 6 to fix w, mnu
Fij_S4_TE = myFij[:nCosParams,:nCosParams] 
Fij_plikHM_lowTEB = Fij_plikHM_lowTEB[:nCosParams,:nCosParams]
Fij_plikHM_TTTEEE_lowTEB = Fij_plikHM_TTTEEE_lowTEB[:nCosParams,:nCosParams]
print Fij_S4_TE.shape, Fij_plikHM_lowTEB.shape, Fij_plikHM_TTTEEE_lowTEB.shape

# convert As to 10^9 As
Fij_S4_TE[:,AsIndex] *= 1e-9
Fij_S4_TE[AsIndex,:] *= 1e-9

# combine using appropriate fsky weighting
fsky_S4 = 0.4
fsky_Pl = 0.2 # this is in addition to area covered by S4

# fsky is in denominator of covariance matrix, putting it in numerator of Fij matrix
Fij_Sum = (Fij_S4_TE+Fij_plikHM_lowTEB)*fsky_S4 + Fij_plikHM_TTTEEE_lowTEB*fsky_Pl
Fij_Sum_Pl = Fij_plikHM_TTTEEE_lowTEB*(fsky_S4+fsky_Pl)

adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lmax for TT... 
adjusting lm

In [14]:
# add sigma = 0.01 prior on tau
tauIndex = 5
tauPrior = 1/0.01**2
Fij_Sum[tauIndex,tauIndex] += tauPrior
Fij_Sum_Pl[tauIndex,tauIndex] += tauPrior

In [15]:
# get sigmas
sigmas_Sum = getSigmas(Fij_Sum)
sigmas_Sum_Pl = getSigmas(Fij_Sum_Pl)

In [16]:
# one more conversion before printout and comparison
ombh2Fac = 100
sigmas_Sum[0] *= ombh2Fac
sigmas_Sum_Pl[0] *= ombh2Fac
paramList[0] = '100 ombh2'

In [17]:
# show sigmas
from prettytable import PrettyTable

print 'Planck only and Planck+CMB-S4'
t = PrettyTable(['Parameter', 'Planck','S4+Planck'])
for i,p in enumerate(paramList[:nCosParams]):
    Zi = sigmas_Sum_Pl[i]
    Bi = sigmas_Sum[i]
    #diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi] )
print t

Planck only and Planck+CMB-S4
+-----------+---------+-----------+
| Parameter |  Planck | S4+Planck |
+-----------+---------+-----------+
| 100 ombh2 |  0.0174 |   0.0301  |
|   omch2   | 0.00171 |  0.00292  |
|     H0    |  0.759  |     1.3   |
|  10^9 As  |  0.0393 |   0.0427  |
|     ns    |  0.0048 |  0.00798  |
|    tau    |  0.009  |  0.00931  |
+-----------+---------+-----------+


# that was without kappa.  Now get kappa into the S4 part

In [18]:
print Fobj.covar.shape
print Fobj.dClVecs.shape
print Fobj.lmin,Fobj.lmax

(3, 3, 2999)
(3, 10, 2999)
2 3000


In [19]:
# make kk fisher with noise
nCosParams = Fobj.nCosParams
dClkk = Fobj.dClVecs[0] # the dCl/dkk part
cov_kk2 = Fobj.covar[0,0] # the kk part
Fij_kk2 = np.zeros((nCosParams,nCosParams))
minEllInd = myLmin-Fobj.lmin  # eg. 30-2
for pNum1 in range(nCosParams):
    for pNum2 in range(pNum1,nCosParams):
        # the next line does the summation over ell
        Fij_kk2[pNum1,pNum2] = np.sum(dClkk[pNum1,minEllInd:]*dClkk[pNum2,minEllInd:]/cov_kk2[minEllInd:])
        Fij_kk2[pNum2,pNum1] = Fij_kk2[pNum1,pNum2]
print 'parameters: ',Fobj.paramList[:nCosParams]
print 'kappa-only, noise-included Fisher matrix:'
print Fij_kk2

parameters:  ['ombh2', 'omch2', 'H0', 'As', 'ns', 'tau', 'mnu', 'w', 'wa']
kappa-only, noise-included Fisher matrix:
[[ 1.41550564e+09 -1.57853830e+09  8.96341460e+05 -4.54711017e+16
  -1.16716845e+08  4.17968771e+04  2.61241272e+07  3.67119270e+07
   1.23475453e+07]
 [-1.57853830e+09  1.76073958e+09 -1.00001942e+06  5.07482298e+16
   1.29956674e+08 -4.68120051e+04 -2.91516271e+07 -4.09627844e+07
  -1.37772735e+07]
 [ 8.96341460e+05 -1.00001942e+06  5.68225276e+02 -2.88484967e+13
  -7.35180269e+04  2.68361730e+01  1.65667230e+04  2.32753524e+04
   7.82686040e+03]
 [-4.54711017e+16  5.07482298e+16 -2.88484967e+13  1.46576755e+24
   3.71461133e+15 -1.37566737e+12 -8.41472575e+14 -1.18186831e+15
  -3.97409679e+14]
 [-1.16716845e+08  1.29956674e+08 -7.35180269e+04  3.71461133e+15
   9.94048054e+06 -3.15491039e+03 -2.13949924e+06 -3.01168275e+06
  -1.01460661e+06]
 [ 4.17968771e+04 -4.68120051e+04  2.68361730e+01 -1.37566737e+12
  -3.15491039e+03  1.50357994e+00  7.85354052e+02  1.09889619e

In [20]:
# convert As to 10^9 As
Fij_kk2[:,AsIndex] *= 1e-9
Fij_kk2[AsIndex,:] *= 1e-9

# fix w, mnu
Fij_kk2 = Fij_kk2[:6,:6]

# get sigmas
sigmas_Sum2 = getSigmas(Fij_Sum+Fij_kk2*fsky_S4)

# one more conversion before printout and comparison
sigmas_Sum2[0] *= ombh2Fac

In [21]:
# show sigmas
from prettytable import PrettyTable

print 'Planck only and Planck+CMB-S4'
t = PrettyTable(['Parameter', 'Planck','S4+Planck'])
for i,p in enumerate(paramList[:nCosParams]):
    Zi = sigmas_Sum_Pl[i]
    Bi = sigmas_Sum2[i]
    #diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi] )
print t

Planck only and Planck+CMB-S4
+-----------+---------+-----------+
| Parameter |  Planck | S4+Planck |
+-----------+---------+-----------+
| 100 ombh2 |  0.0174 |   0.0227  |
|   omch2   | 0.00171 |  0.00089  |
|     H0    |  0.759  |   0.442   |
|  10^9 As  |  0.0393 |   0.0319  |
|     ns    |  0.0048 |   0.0033  |
|    tau    |  0.009  |  0.00837  |
+-----------+---------+-----------+


In [22]:
# print out with CMB-S4 table 8.1 values
sigmas_S4book_Planck = [0.017,0.0014,0.7,0.039,0.004,0.01]
sigmas_S4book_PlanckS4 = [0.003,0.0006,0.24,0.021,0.002,0.006]

t = PrettyTable(['Parameter','Planck_book', 'Planck_Z','% diff'])
for i,p in enumerate(paramList[:nCosParams]):
    Bi = sigmas_Sum_Pl[i]
    Zi = sigmas_S4book_Planck[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi,"%5.3g" % diff] )
print t

t = PrettyTable(['Parameter','S4+Planck_book', 'S4+Planck_Z','% diff'])
for i,p in enumerate(paramList[:nCosParams]):
    Bi = sigmas_Sum2[i]
    Zi = sigmas_S4book_PlanckS4[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi,"%5.3g" % diff] )
print t

+-----------+-------------+----------+--------+
| Parameter | Planck_book | Planck_Z | % diff |
+-----------+-------------+----------+--------+
| 100 ombh2 |    0.017    |  0.0174  |  2.61  |
|   omch2   |    0.0014   | 0.00171  |  21.8  |
|     H0    |      0.7    |  0.759   |  8.38  |
|  10^9 As  |    0.039    |  0.0393  |  0.69  |
|     ns    |    0.004    |  0.0048  |  20.1  |
|    tau    |     0.01    |  0.009   | -9.98  |
+-----------+-------------+----------+--------+
+-----------+----------------+-------------+--------+
| Parameter | S4+Planck_book | S4+Planck_Z | % diff |
+-----------+----------------+-------------+--------+
| 100 ombh2 |     0.003      |    0.0227   |   655  |
|   omch2   |     0.0006     |   0.00089   |  48.3  |
|     H0    |      0.24      |    0.442    |  84.4  |
|  10^9 As  |     0.021      |    0.0319   |  52.1  |
|     ns    |     0.002      |    0.0033   |  64.8  |
|    tau    |     0.006      |   0.00837   |  39.5  |
+-----------+----------------+----

# Next try with CosmicFish results

In [23]:
cfDir = 'cosmicfish_results/'
FijFilename = '3_CMBS4_TEB_fisher_matrix_cls.dat'  # TEB only

# The parameters of this Fisher matrix are:
#
#    1    omegabh2    \Omega_b h^2       0.2220000000000000E-01
#    2    omegach2    \Omega_c h^2       0.1200000000000000E+00
#    3    h    h       0.6900000000000001E+00
#    4    logA    {\rm{ln}}(10^{10} A_s)       0.3091042453358316E+01
#    5    ns    n_s       0.9660000000000000E+00
#    6    tau    \tau       0.6000000000000000E-01
#

Fij_CF = np.loadtxt(cfDir+FijFilename)
print Fij_CF

[[ 4.30400590e+09 -6.59605228e+08 -1.95219475e+08  7.37773638e+07
   9.94588306e+07 -1.42641941e+08]
 [-6.59605228e+08  2.06770781e+08  5.40309866e+07 -1.12167620e+07
  -1.24332342e+07  2.26744356e+07]
 [-1.95219475e+08  5.40309866e+07  1.54081445e+07 -3.36576138e+06
  -3.86681499e+06  6.28546909e+06]
 [ 7.37773638e+07 -1.12167620e+07 -3.36576138e+06  3.44653161e+06
   3.14441968e+06 -6.42453437e+06]
 [ 9.94588306e+07 -1.24332342e+07 -3.86681499e+06  3.14441968e+06
   3.66513392e+06 -5.99246391e+06]
 [-1.42641941e+08  2.26744356e+07  6.28546909e+06 -6.42453437e+06
  -5.99246391e+06  1.22193648e+07]]


# Transform to $10^9 As$ via Jacobian

In [24]:
# only one partial derivative needed
# d( ln(10^10*As)) / d(10^9*As) = 1/(10^9*As)*d(10*10^9*As)/d(10^9*As) = 1/(10^9*As)
TenToTheNineAs_fid = 2.2
AsIndex = 3
Fij_CF[:,AsIndex] *= 1/TenToTheNineAs_fid
Fij_CF[AsIndex,:] *= 1/TenToTheNineAs_fid

# Transform to $H_0$ via Jacobian

In [25]:
# d(h)/d(100h) = 1/100
HIndex = 2
Fij_CF[:,HIndex] /= 100
Fij_CF[HIndex,:] /= 100

In [26]:
# get sigmas
getSigmas = lambda Fij: np.sqrt(np.diag(np.linalg.inv(Fij)))
sigmas_CosmicFish = getSigmas(Fij_CF)
print sigmas_CosmicFish

[3.83938345e-05 8.56005710e-04 3.23719005e-01 2.75374827e-02
 2.48307919e-03 7.44515332e-03]


In [27]:
# combine with others
# fsky is in denominator of covariance matrix, putting it in numerator of Fij matrix
#Fij_Sum = (Fij_S4_TE+Fij_plikHM_lowTEB)*fsky_S4 + Fij_plikHM_TTTEEE_lowTEB*fsky_Pl
#Fij_Sum_Pl = Fij_plikHM_TTTEEE_lowTEB*(fsky_S4+fsky_Pl)

# fsky already applied to Fij_CF
Fij_Sum_CF = Fij_CF + Fij_plikHM_TTTEEE_lowTEB*fsky_Pl


In [28]:
# add sigma = 0.01 prior on tau
tauIndex = 5
tauPrior = 1/0.01**2
#Fij_Sum[tauIndex,tauIndex] += tauPrior
#Fij_Sum_Pl[tauIndex,tauIndex] += tauPrior

Fij_Sum_CF[tauIndex,tauIndex] += tauPrior


# Transform to $100 \Omega_b h^2$ via Jacobian

In [29]:
# d(ombh2)/d(100obmh2) = 1/100
ObIndex = 0
Fij_CF[:,ObIndex] /= 100
Fij_CF[ObIndex,:] /= 100

Fij_Sum_CF[:,ObIndex] /= 100
Fij_Sum_CF[ObIndex,:] /= 100

Fij_kk2[:,ObIndex] /= 100
Fij_kk2[ObIndex,:] /= 100

In [30]:
# Show the sigmas
# get sigmas
#sigmas_Sum = getSigmas(Fij_Sum)
#sigmas_Sum_Pl = getSigmas(Fij_Sum_Pl)
sigmas_Sum_CF = getSigmas(Fij_Sum_CF)

t = PrettyTable(['Parameter','Planck_book', 'Planck_Z','% diff'])
for i,p in enumerate(paramList[:nCosParams]):
    Bi = sigmas_Sum_Pl[i]
    Zi = sigmas_S4book_Planck[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi,"%5.3g" % diff] )
print t

t = PrettyTable(['Parameter','S4+Planck_book', 'S4_CF+Planck_Z','% diff'])
for i,p in enumerate(paramList[:nCosParams]):
    Bi = sigmas_Sum_CF[i]
    Zi = sigmas_S4book_PlanckS4[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi,"%5.3g" % diff] )
print t
print "This from CosmicFish with no kappa"

+-----------+-------------+----------+--------+
| Parameter | Planck_book | Planck_Z | % diff |
+-----------+-------------+----------+--------+
| 100 ombh2 |    0.017    |  0.0174  |  2.61  |
|   omch2   |    0.0014   | 0.00171  |  21.8  |
|     H0    |      0.7    |  0.759   |  8.38  |
|  10^9 As  |    0.039    |  0.0393  |  0.69  |
|     ns    |    0.004    |  0.0048  |  20.1  |
|    tau    |     0.01    |  0.009   | -9.98  |
+-----------+-------------+----------+--------+
+-----------+----------------+----------------+--------+
| Parameter | S4+Planck_book | S4_CF+Planck_Z | % diff |
+-----------+----------------+----------------+--------+
| 100 ombh2 |     0.003      |    0.00372     |  23.9  |
|   omch2   |     0.0006     |    0.000675    |  12.5  |
|     H0    |      0.24      |     0.256      |  6.84  |
|  10^9 As  |     0.021      |     0.0214     |   1.8  |
|     ns    |     0.002      |    0.00213     |   6.3  |
|    tau    |     0.006      |    0.00575     | -4.15  |
+------

In [31]:
# now add in kappa
Fij_Sum_CF_k = Fij_Sum_CF + Fij_kk2*fsky_S4
sigmas_Sum_CF_k = getSigmas(Fij_Sum_CF_k)

In [32]:
t = PrettyTable(['Parameter','Planck_book', 'Planck_Z','% diff'])
for i,p in enumerate(paramList[:nCosParams]):
    Bi = sigmas_Sum_Pl[i]
    Zi = sigmas_S4book_Planck[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi,"%5.3g" % diff] )
print t

t = PrettyTable(['Parameter','S4+Planck_book', 'S4_CF+Planck_Z+kk_Z','% diff'])
for i,p in enumerate(paramList[:nCosParams]):
    Bi = sigmas_Sum_CF_k[i]
    Zi = sigmas_S4book_PlanckS4[i]
    diff = (Bi-Zi)/Zi *100
    t.add_row([p,"%5.3g" % Zi,"%5.3g" % Bi,"%5.3g" % diff] )
print t

+-----------+-------------+----------+--------+
| Parameter | Planck_book | Planck_Z | % diff |
+-----------+-------------+----------+--------+
| 100 ombh2 |    0.017    |  0.0174  |  2.61  |
|   omch2   |    0.0014   | 0.00171  |  21.8  |
|     H0    |      0.7    |  0.759   |  8.38  |
|  10^9 As  |    0.039    |  0.0393  |  0.69  |
|     ns    |    0.004    |  0.0048  |  20.1  |
|    tau    |     0.01    |  0.009   | -9.98  |
+-----------+-------------+----------+--------+
+-----------+----------------+---------------------+--------+
| Parameter | S4+Planck_book | S4_CF+Planck_Z+kk_Z | % diff |
+-----------+----------------+---------------------+--------+
| 100 ombh2 |     0.003      |       0.00322       |  7.41  |
|   omch2   |     0.0006     |       0.000569      |  -5.2  |
|     H0    |      0.24      |        0.213        | -11.1  |
|  10^9 As  |     0.021      |        0.0202       | -3.92  |
|     ns    |     0.002      |       0.00195       | -2.55  |
|    tau    |     0.006 